In [1]:
%pip install torch
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from matplotlib import pyplot as plt

In [2]:
# データの読み込みと前処理
csv_path = 'C:\\Users\\admin\\Documents\\testB\\data\\train.csv'
data_dir = 'C:\\Users\\admin\\Documents\\testB\\data\\train\\train'

df = pd.read_csv(csv_path)
df_filtered = df[df['gender_status'] != 5]

def center_crop_and_resize(img, size):
    h, w, _ = img.shape
    min_dim = min(h, w)
    start_x = (w - min_dim) // 2
    start_y = (h - min_dim) // 2
    cropped_img = img[start_y:start_y + min_dim, start_x:start_x + min_dim]
    resized_img = cv2.resize(cropped_img, (size, size))
    return resized_img

x_data = []
y_data = []

for index, row in df_filtered.iterrows():
    img_path = os.path.join(data_dir, row['image'])
    img = cv2.imread(img_path)
    if img is not None:
        img = center_crop_and_resize(img, 224)
        x_data.append(img)
        y_data.append(row['gender_status'])
    else:
        print(f"画像が見つからないか、読み込めませんでした: {img_path}")

x_data = np.array(x_data)
y_data = np.array(y_data)

In [3]:
# 各カテゴリの画像の枚数を数える
category_counts = df['gender_status'].value_counts()

# 結果を表示
for category, count in category_counts.items():
    print(f"カテゴリ {category} の画像枚数: {count}")

カテゴリ 1 の画像枚数: 1294
カテゴリ 0 の画像枚数: 1210
カテゴリ 4 の画像枚数: 834
カテゴリ 2 の画像枚数: 375
カテゴリ 3 の画像枚数: 371
カテゴリ 7 の画像枚数: 100
カテゴリ 6 の画像枚数: 53
カテゴリ 5 の画像枚数: 1


In [4]:
# クラスの不均等調整（クラス5の重みを無視）
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_data),  # クラス5も含むクラス全体を指定
    y=y_data
)

# クラス5の重みをゼロに設定
class_weights = np.array(class_weights)  # Numpy配列に変換
#class_weights[5] = 0  # クラス5の重みをゼロに設定
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Tensor形式に変換してPyTorchに渡す
weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
weights_tensor = weights_tensor.to(device)
# 損失関数の設定
criterion = torch.nn.CrossEntropyLoss(weight=weights_tensor)

print("Class Weights Dictionary:", class_weights_dict)

Using device: cpu
Class Weights Dictionary: {0: np.float64(0.5002361275088548), 1: np.float64(0.4677633031574299), 2: np.float64(1.614095238095238), 3: np.float64(1.6314978821717365), 4: np.float64(0.7257622473449812), 5: np.float64(11.420485175202156), 6: np.float64(6.0528571428571425)}


In [5]:
#trainデータとtestデータをシャッフルしてクラスの均等性を揃える
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.3, random_state=42,stratify=y_data)

# PyTorch Dataset作成
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        image = self.x_data[idx]
        label = self.y_data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [6]:
# データ拡張と変換
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(360),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor()
])

train_dataset = CustomDataset(x_train, y_train, transform=transform)
val_dataset = CustomDataset(x_val, y_val, transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
]))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [7]:
# モデルの定義
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = models.resnet50(pretrained=True)
for param in base_model.parameters():
    param.requires_grad = False

base_model.fc = nn.Sequential(
    nn.Linear(base_model.fc.in_features, 256),
    #nn.ReLU(),
    #nn.Dropout(0.25),
    nn.Linear(256, 7),
    nn.Softmax(dim=1)
)

base_model = base_model.to(device)

criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights)).to(device))
optimizer = optim.Adam(base_model.fc.parameters(), lr=0.0001)

c:\Users\admin\anaconda3\envs\testB\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\admin\anaconda3\envs\testB\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
# トレーニングループの定義
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10):
    for epoch in range(epochs):
        # モデルをトレーニングモードに設定
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        # トレーニングデータを処理
        for images, labels in train_loader:
            images = images.float()
            labels = labels.float()

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_accuracy = 100 * train_correct / train_total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

        # モデルを検証モードに設定
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# トレーニングループの実行
train_model(base_model, train_loader, val_loader, criterion, optimizer, device, epochs=10)

RuntimeError: expected scalar type Float but found Double

In [19]:
import torch

# トレーニングループの定義
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10):
    for epoch in range(epochs):
        # モデルをトレーニングモードに設定
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        # トレーニングデータを処理
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(torch.long)  # 交差エントロピー損失関数は `Long` 型を期待

            optimizer.zero_grad()

            outputs = model(images).to(torch.float32)  # `torch.float32` に変換
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)  # 修正
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_accuracy = 100 * train_correct / train_total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

        # モデルを検証モードに設定
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)  # デバイスに送る

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)  # 修正
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# トレーニングループの実行
train_model(base_model, train_loader, val_loader, criterion, optimizer, device, epochs=10)

RuntimeError: expected scalar type Float but found Double

In [ ]:
# エポック数
epochs = range(1, len(train_losses) + 1)

# 損失のプロット
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'ro-', label='Training loss')
plt.plot(epochs, val_losses, 'bo-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 精度のプロット
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'ro-', label='Training accuracy')
plt.plot(epochs, val_accuracies, 'bo-', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# プロットの表示
plt.tight_layout()
plt.show()


In [ ]:
# テストデータのディレクトリパスを設定
test_data_dir = 'C:\\Users\\adomin\\Documents\\testB\\data\\test\\test'

# テストデータの画像を読み込み、前処理
x_test = []
image_names = []

for img_name in os.listdir(test_data_dir):
    img_path = os.path.join(test_data_dir, img_name)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (224, 224))
        x_test.append(img)
        image_names.append(img_name)
    else:
        print(f"画像が見つからないか、読み込めませんでした: {img_path}")

# PyTorch Tensorへの変換
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

x_test_tensor = torch.stack([transform(img) for img in x_test])  # テストデータをTensor形式に変換
x_test_tensor = x_test_tensor.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# 予測を行う
base_model.eval()  # モデルを評価モードに切り替え
predictions = []

with torch.no_grad():  # 勾配計算を無効化
    for img_tensor in x_test_tensor:
        img_tensor = img_tensor.unsqueeze(0)  # バッチサイズ1に対応させる
        output = base_model(img_tensor)
        predicted_class = torch.argmax(output, dim=1).item()
        predictions.append(predicted_class)

# 結果をDataFrameにまとめる
results_df = pd.DataFrame({'image': image_names, 'gender_status': predictions})

# 結果をCSVファイルに保存
results_csv_path = 'C:\\Users\\adomin\\Documents\\testB\\data\\predictions8.csv'
results_df.to_csv(results_csv_path, index=False)

print(f"予測結果が {results_csv_path} に保存されました。")
